In [ ]:
%%writefile Granos2D.cpp

#include <iostream>
#include <cmath>
#include "vector.h"
#include "Random64.h"
using namespace std;


// Constantes del problema físico:
const double g=9.8;
const int Nx=5, Ny=5;
const int N=Nx*Ny, Ntot=N+4;  // Número total de granos (contando las paredes)
const double KHertz=1.0e4, Gamma=15, kCundall=500, mu=0.4;
double Lx=60, Ly=60;  // Tamaño de la caja

// Constantes del algoritmo de integración
const double Xi = 0.1786178958448091;
const double Lambda = -0.2123418310626054;
const double Chi = -0.06626458266981849;
const double Um2LambdaU2 = (1-2*Lambda)/2;
const double Um2ChipXi = 1-2*(Chi+Xi);

// Declaración de las clases:
class Cuerpo;
class Colisionador;

// Declaración de las interfases de las clases:
class Cuerpo{
  private:
    vector3D r,V,F; double m,R; double theta,omega,tau,I;
  public:
    void Inicie(double x0,double y0,double Vx0,double Vy0,double theta0,double omega0,
	        double m0,double R0);
    void BorreFuerza(void){F.load(0,0,0); tau=0;}; // Inline
    void SumeFuerza(vector3D dF, double dtau){F+=dF; tau+=dtau;}; // Inline
    void Mueva_r(double dt, double coef);
    void Mueva_v(double dt, double coef);
    void Dibujese(void);
    double Getx(void){return r.x();}; // Inline
    double Gety(void){return r.y();}; // Inline
    friend class Colisionador;    // De esta manera, hacemos que la clase Colisionador pueda tener acceso a los datos privados de la clase Cuerpo
};
class Colisionador{
  private:
    double xCundall[Ntot][Ntot], sold[Ntot][Ntot];
  public:
    void Inicie(void);
    void CalculeTodasLasFuerzas(Cuerpo * Granos, double dt);
    void CalculeFuerzaEntre(Cuerpo & Grano1, Cuerpo & Grano2, double & xCundall, double & sold, double dt);
};

// Implementación de las funciones de las clases:
// Funciones de la clase Cuerpo:
void Cuerpo::Inicie(double x0,double y0,double Vx0,double Vy0,double theta0, double omega0,
	      double m0,double R0){
  r.load(x0,y0,0); V.load(Vx0,Vy0,0); m=m0; R=R0;
  theta=theta0; omega=omega0; I=2.0/5.0*m*R*R;
}
void Cuerpo::Mueva_r(double dt, double coef){
  // Algoritmo de Forest-Ruth
  r+=(coef*dt)*V; theta+=omega*(coef*dt);
}
void Cuerpo::Mueva_v(double dt, double coef){
  // Algoritmo de Forest-Ruth
  V+=(coef*dt/m)*F; omega+=tau*(coef*dt/I);
}
void Cuerpo::Dibujese(void){
  cout<<" , "<<r.x()<<"+"<<R<<"*cos(t),"<<r.y()<<"+"<<R<<"*sin(t), "
      <<r.x()<<"+"<<R*cos(theta)/7.0<<"*t,"<<r.y()<<"+"<<R*sin(theta)/7.0<<"*t";  // Con este comando, se dibuja una línea en cada grano para poder observar su rotación
}

// Funciones de la clase Colisionador:
void Colisionador::Inicie(void){
  int i,j;
  for(i=0;i<Ntot;i++)
    for(j=0;j<Ntot;j++)
      xCundall[i][j]=sold[i][j]=0;
}
void Colisionador::CalculeTodasLasFuerzas(Cuerpo * Grano, double dt){
  int i,j;
  // 1) Borramos las fuerzas de todos los granos:
  for(i=0;i<N+4;i++)
    Grano[i].BorreFuerza();
  // Sumo la fuerza de la gravedad:
  vector3D Fg;
  for(i=0;i<N;i++){
    Fg.load(0,-Grano[i].m*g,0);
    Grano[i].SumeFuerza(Fg,0);
  }
  // 2) Se recorre por parejas la clase Cuerpo creada, se calcula la fuerza de cada pareja y se suma a cada grano:
  for(i=0;i<N;i++){
    for(j=i+1;j<N+4;j++){
      CalculeFuerzaEntre(Grano[i],Grano[j],xCundall[i][j],sold[i][j],dt);
    }
  }

}
void Colisionador::CalculeFuerzaEntre(Cuerpo & Grano1, Cuerpo & Grano2, double & xCundall, double & sold, double dt){
  // Determinamos si hay colisión:
  vector3D r21 = Grano2.r-Grano1.r; double r = r21.norm();
  double R1 = Grano1.R, R2 = Grano2.R;
  double s=(R1+R2)-r;
  // Condición de colisión:
  if(s>0){
    // Calculamos el vector normal:
    vector3D n = (1.0/r)*r21,t,k;  t.load(n.y(),-n.x(),0); k.load(0,0,1);
    // Calculamos la velocidad de contacto:
    vector3D Rw;  Rw.load(0,0,R2*Grano2.omega+R1*Grano1.omega);
    vector3D Vc=(Grano2.V-Grano1.V)-(Rw^n);
    vector3D Vcn=(Vc*n)*n,  Vct=Vc-Vcn;
    // Calculamos la fuerza normal (Hertz-Kuramoto-Kano):
    double m1=Grano1.m, m2=Grano2.m;  double m21=m1*m2/(m1+m2);
    double Fn = (KHertz*pow(s,1.5))+(-Gamma*sqrt(s)*m21)*(Vc*n);
    // Calculamos la fuerza tangencial (Cundall):
    xCundall+=Vct*t*dt; double Ft=-kCundall*xCundall; double Ftmax=mu*fabs(Fn);
    if(fabs(Ft)>Ftmax)
      Ft=Ft/fabs(Ft)*Ftmax;
    // Cargamos las fuerzas por componentes:
    vector3D F1,F2,tau1,tau2;
    F2=n*Fn+t*Ft; F1=F2*(-1); tau2=((n*(-R2))^F2);  tau1=((n*R1)^F1);
    Grano2.SumeFuerza(F2,tau2*k);  Grano1.SumeFuerza(F1,tau1*k);
  }
  if(sold>=0 && s<0)
    xCundall=0;
  sold=s;
}
//
//----------- Funciones Globales -----------

//Funciones de animación:
void InicieAnimacion(void){
  //cout<<"set terminal gif animate"<<end;
  //cout<<"set output 'Gas2D.gif'"<<end;
  cout<<"unset key"<<endl;
  cout<<"set xrange[-10:"<<Lx+10<<"]"<<endl;
  cout<<"set yrange[-10:"<<Ly+10<<"]"<<endl;
  cout<<"set size ratio -1"<<endl;
  cout<<"set parametric"<<endl;
  cout<<"set trange[0:7]"<<endl;
  cout<<"set isosamples 12"<<endl;
}

void InicieCuadro(void){
  cout<<"plot 0,0 ";
  cout<<" , "<<Lx/7<<"*t,0";  // Pared de abajo
  cout<<" , "<<Lx/7<<"*t,"<<Ly;  // Pared de arriba
  cout<<" , 0,"<<Ly/7<<"*t";  // Pared de la izquierda
  cout<<" , "<<Lx<<","<<Ly/7<<"*t";  // Pared de la derecha
}

void TermineCuadro(void){
  cout<<endl;
}

int main(){

  Cuerpo Grano[Ntot];  // Tendremos un gas de N granos, pero las 4 que añadimos de manera extra van a sernos útiles para definir las paredes de la caja, y por lo tanto, habrá interacción entre la caja y el gas
  Colisionador HertzKuramotoKano;
  Crandom ran64(12);
  int i,j;
  // Parámetros de la simulación:
  double m0=1.0, R0=2.0;
  // Variables auxiliares para la condición inicial y correr la simulación:
  double k_BT=10, V0=0,/*sqrt(k_BT/m0)*/ omega0=2*M_PI; // Definimos una temperatura y, mediante el teorema del virial, definimos una velocidad inicial
  double t, dt=1e-3, ttotal=20/*10*(Lx/V0)*/;
  int Ncuadros=1000; double tdibujo,tcuadro=ttotal/Ncuadros;
  double dx=Lx/(Nx+1), dy=Ly/(Ny+1);
  double theta;
  double x0,y0,Vx0,Vy0;
  // Variables auxiliares para las paredes:
  double Rpared=100*Lx, Mpared=100*m0;

  InicieAnimacion();
  // INICIO:
  // Inicializamos las paredes:
  //----------------(x0,y0,Vx0,Vy0,theta0,omega0,m0,R0)
  Grano[N].Inicie(Lx/2, Ly+Rpared, 0, 0, 0, 0, Mpared, Rpared); // Pared de arriba
  Grano[N+1].Inicie(Lx/2, -Rpared, 0, 0, 0, 0, Mpared, Rpared); // Pared de abajo
  Grano[N+2].Inicie(Lx+Rpared, Ly/2, 0, 0, 0, 0, Mpared, Rpared); // Pared de la derecha
  Grano[N+3].Inicie(-Rpared, Ly/2, 0, 0, 0, 0, Mpared, Rpared); // Pared de la izquierda
  // Inicializamos los granos:
  for(i=0;i<Nx;i++)
    for(j=0;j<Ny;j++){
      theta=2*M_PI*ran64.r();
      x0=(i+1)*dx; y0=(j+1)*dy; Vx0=V0*cos(theta); Vy0=V0*sin(theta);
      //----------------(x0,y0,Vx0,Vy0,theta0,omega0,m0,R0)
      Grano[j*Nx+i].Inicie(x0, y0, Vx0, Vy0, 0, omega0, m0, R0);
  }
  // CORRE PROGRAMA:
  for(t=tdibujo=0;t<ttotal;t+=dt,tdibujo+=dt){
    if(tdibujo>tcuadro){
      //cout<<Grano[0].Getx()<<" "<<Grano[0].Gety()<<endl;
      //cout<<Grano[1].Getx()<<" "<<Grano[1].Gety()<<endl;
      InicieCuadro();
      for(i=0;i<N;i++) Grano[i].Dibujese();
      TermineCuadro();
      tdibujo=0;
    }
    /* Ahora el cálculo de las fuerzas depende de la clase colisionador y no de la clase cuerpo. Además, recordemos que estamos
       aplicando el método de integración PEFRL de manera simultánea para los N cuerpos que tengamos.*/

    for(i=0;i<N;i++) Grano[i].Mueva_r(dt,Xi);
    HertzKuramotoKano.CalculeTodasLasFuerzas(Grano,dt);  for(i=0;i<N;i++) Grano[i].Mueva_v(dt,Um2LambdaU2);
    for(i=0;i<N;i++) Grano[i].Mueva_r(dt,Chi);
    HertzKuramotoKano.CalculeTodasLasFuerzas(Grano,dt);  for(i=0;i<N;i++) Grano[i].Mueva_v(dt,Lambda);
    for(i=0;i<N;i++) Grano[i].Mueva_r(dt,Um2ChipXi);
    HertzKuramotoKano.CalculeTodasLasFuerzas(Grano,dt);  for(i=0;i<N;i++) Grano[i].Mueva_v(dt,Lambda);
    for(i=0;i<N;i++) Grano[i].Mueva_r(dt,Chi);
    HertzKuramotoKano.CalculeTodasLasFuerzas(Grano,dt);  for(i=0;i<N;i++) Grano[i].Mueva_v(dt,Um2LambdaU2);
    for(i=0;i<N;i++) Grano[i].Mueva_r(dt,Xi);

  }
  return 0;
}

Overwriting Granos2D.cpp


In [ ]:
%%shell
g++ Granos2D.cpp
./a.out > "Instrucciones - Granos2D.txt"